# Reconciling Scale Space and Stack Stages

See Evernote notebook with a similar name.

Here we are trying to reconcile our development so far with our work on scale representations. This starts from the observation that an eigenvector of ```[1, 1, 1, 1]``` is an average filter.

## Comparing Downsampling and VPU

When we downsample, one naive way of doing this is averaging the inputs - i.e. times by 1, sum, then divide by the length.

In [1]:
from src.tests.test_vpu import rand_same, rand_diff, rand_opposite
from src.var_processor.vpu import BinaryVPU
import numpy as np

In [2]:
# Generate a VPU of length 4 and initialise to have equal eigenvector
vpu = BinaryVPU(4)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(size=4), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (1/np.sqrt(4))*127)

[[70 70 70 70]
 [70 70 70 70]
 [70 70 70 70]
 [70 70 70 70]] [[0.5511811 0.5511811 0.5511811 0.5511811]
 [0.5511811 0.5511811 0.5511811 0.5511811]
 [0.5511811 0.5511811 0.5511811 0.5511811]
 [0.5511811 0.5511811 0.5511811 0.5511811]]
[[63]
 [63]
 [63]
 [63]] 63.5


In [3]:
# All ones should give a one output
ones = np.ones(4).reshape(-1, 1)
vpu.forward(ones)

array([[1]], dtype=int8)

In [4]:
# All zeros should give a zero output
zeros = np.zeros(4).reshape(-1, 1)
vpu.forward(zeros)

array([[0]], dtype=int8)

In [6]:
# Have a look at some random inputs and outputs
for i in range(100):
    # Generate ternary random array of length 4
    rand = np.random.randint(low=-1, high=2, size=(4,1), dtype=np.int8)
    r = vpu.forward(rand)
    print(rand, r, "---", sep="\n")

[[ 1]
 [ 0]
 [-1]
 [ 0]]
[[0]]
---
[[-1]
 [ 0]
 [-1]
 [ 0]]
[[-1]]
---
[[-1]
 [ 0]
 [ 0]
 [ 0]]
[[-1]]
---
[[-1]
 [ 0]
 [ 1]
 [ 0]]
[[0]]
---
[[-1]
 [-1]
 [ 1]
 [ 0]]
[[0]]
---
[[0]
 [1]
 [1]
 [1]]
[[1]]
---
[[ 0]
 [ 1]
 [ 0]
 [-1]]
[[0]]
---
[[ 0]
 [ 1]
 [ 0]
 [-1]]
[[0]]
---
[[1]
 [0]
 [1]
 [0]]
[[1]]
---
[[0]
 [0]
 [1]
 [0]]
[[0]]
---
[[-1]
 [-1]
 [ 0]
 [ 0]]
[[-1]]
---
[[ 1]
 [-1]
 [ 0]
 [ 0]]
[[0]]
---
[[ 0]
 [ 1]
 [ 1]
 [-1]]
[[0]]
---
[[ 1]
 [ 1]
 [-1]
 [ 0]]
[[0]]
---
[[-1]
 [ 1]
 [ 0]
 [ 0]]
[[0]]
---
[[0]
 [1]
 [0]
 [0]]
[[0]]
---
[[-1]
 [-1]
 [-1]
 [ 1]]
[[-1]]
---
[[0]
 [1]
 [0]
 [1]]
[[1]]
---
[[-1]
 [ 0]
 [ 1]
 [-1]]
[[-1]]
---
[[ 0]
 [ 1]
 [-1]
 [ 0]]
[[0]]
---
[[0]
 [0]
 [1]
 [0]]
[[1]]
---
[[-1]
 [ 0]
 [ 0]
 [ 1]]
[[0]]
---
[[ 0]
 [ 0]
 [ 0]
 [-1]]
[[-1]]
---
[[-1]
 [ 0]
 [ 0]
 [ 0]]
[[-1]]
---
[[0]
 [0]
 [0]
 [1]]
[[1]]
---
[[ 1]
 [-1]
 [-1]
 [-1]]
[[0]]
---
[[1]
 [1]
 [1]
 [1]]
[[1]]
---
[[1]
 [1]
 [1]
 [1]]
[[1]]
---
[[ 0]
 [ 0]
 [ 1]
 [-1]]
[[0]]
---
[[1]
 [0]
 [1]

If we were downsampling, a signal of ```[50, 100, 50, 100]``` would be downsampled to 75. Does this behaviour occur with our VPU?

We'd need to sum over 100 iterations with 0 mean.

In [9]:
from src.var_processor.pb_threshold import signal_pre_processor

signal = np.asarray([50, 100, 50, 100]).reshape(-1, 1)
data_summed = np.zeros(shape=(4, 1))
r_summed = 0
for i in range(100):
    data_in = signal_pre_processor(signal, zeros)
    r = vpu.forward(data_in)
    data_summed += data_in
    r_summed += r.item()

data_summed, r_summed

(array([[100.],
        [100.],
        [100.],
        [100.]]), 99)

Ah - the signal pre processor does not work with a zero mean - the random values for comparison are always 0 and the symmetrical range either side of the mean is zero.

Let's try with 128 as the mean.

In [10]:
data_summed = np.zeros(shape=(4, 1))
r_summed = 0
for i in range(100):
    data_in = signal_pre_processor(signal, ones*128)
    r = vpu.forward(data_in)
    data_summed += data_in
    r_summed += r.item()

data_summed, r_summed

(array([[-65.],
        [-23.],
        [-53.],
        [-13.]]), -60)

In [12]:
# Then we add back in the mean.
print(data_summed+128, r_summed+128)

[[ 63.]
 [105.]
 [ 75.]
 [115.]] 68


In [21]:
data_summed = np.zeros(shape=(4, 1))
r_summed = 0
for i in range(500):
    data_in = signal_pre_processor(signal, ones*128)
    r = vpu.forward(data_in)
    data_summed += data_in
    r_summed += r.item()
recon_data = (data_summed/500)*128 + 128
recon_r = (r_summed/500)*128 + 128
print(recon_data, recon_r)

[[46.848]
 [97.792]
 [49.664]
 [98.816]] 50.688


r is 50 not 75.

In [42]:
# Try with a random vector
signal = np.random.randint(255, size=(4,1))
data_summed = np.zeros(shape=(4, 1))
r_summed = 0
for i in range(500):
    data_in = signal_pre_processor(signal, ones*128)
    r = vpu.forward(data_in)
    data_summed += data_in
    r_summed += r.item()
recon_data = (data_summed/500)*128 + 128
recon_r = (r_summed/500)*128 + 128
print(signal, recon_data, recon_r, signal.mean(), sep="\n----\n")

[[ 54]
 [ 87]
 [235]
 [104]]
----
[[ 54.784]
 [ 83.456]
 [240.384]
 [102.4  ]]
----
124.416
----
120.0


It is an approximation to the mean.

Let's have a look at differences.

In [47]:
# Try with a random vector
signal = np.random.randint(255, size=(4,1))
data_summed = np.zeros(shape=(4, 1))
r_summed = 0
pred_summed = np.zeros(shape=(4, 1))
diff_summed = np.zeros(shape=(4, 1))

for i in range(500):
    data_in = signal_pre_processor(signal, ones*128)
    # Forward pass
    r = vpu.forward(data_in)
    # Backward pass
    preds = vpu.backward(r)
    diffs = data_in - preds
    # Add to totals
    data_summed += data_in
    r_summed += r.item()
    pred_summed += preds
    diff_summed += diffs
recon_data = (data_summed/500)*128 + 128
recon_r = (r_summed/500)*128 + 128
recon_preds = (pred_summed/500)*128 + 128
recon_diffs = (diff_summed/500)*128
print(signal, recon_data, recon_r, signal.mean(), recon_preds, recon_diffs, signal-signal.mean(), sep="\n----\n")

[[ 25]
 [180]
 [165]
 [210]]
----
[[ 25.856]
 [179.712]
 [162.048]
 [213.504]]
----
144.89600000000002
----
145.0
----
[[144.896]
 [144.896]
 [144.384]
 [144.384]]
----
[[-119.04 ]
 [  34.816]
 [  17.664]
 [  69.12 ]]
----
[[-120.]
 [  35.]
 [  20.]
 [  65.]]


In [48]:
-127+3*127

254

In [49]:
127+3*-127

-254